--- Day 14: Extended Polymerization ---

The incredible pressures at this depth are starting to put a strain on your submarine. The submarine has polymerization equipment that would produce suitable materials to reinforce the submarine, and the nearby volcanically-active caves should even have the necessary input elements in sufficient quantities.

The submarine manual contains instructions for finding the optimal polymer formula; specifically, it offers a polymer template and a list of pair insertion rules (your puzzle input). You just need to work out what polymer would result after repeating the pair insertion process a few times.

For example:

```
NNCB

CH -> B
HH -> N
CB -> H
NH -> C
HB -> C
HC -> B
HN -> C
NN -> C
BH -> H
NC -> B
NB -> B
BN -> B
BB -> N
BC -> B
CC -> N
CN -> C
```

The first line is the polymer template - this is the starting point of the process.

The following section defines the pair insertion rules. A rule like AB -> C means that when elements A and B are immediately adjacent, element C should be inserted between them. These insertions all happen simultaneously.

So, starting with the polymer template NNCB, the first step simultaneously considers all three pairs:

-    The first pair (NN) matches the rule NN -> C, so element C is inserted between the first N and the second N.
-    The second pair (NC) matches the rule NC -> B, so element B is inserted between the N and the C.
-    The third pair (CB) matches the rule CB -> H, so element H is inserted between the C and the B.

Note that these pairs overlap: the second element of one pair is the first element of the next pair. Also, because all pairs are considered simultaneously, inserted elements are not considered to be part of a pair until the next step.

After the first step of this process, the polymer becomes NCNBCHB.

Here are the results of a few steps using the above rules:

```
Template:     NNCB
After step 1: NCNBCHB
After step 2: NBCCNBBBCBHCB
After step 3: NBBBCNCCNBBNBNBBCHBHHBCHB
After step 4: NBBNBNBBCCNBCNCCNBBNBBNBBBNBBNBBCBHCBHHNHCBBCBHCB
```

This polymer grows quickly. After step 5, it has length 97; After step 10, it has length 3073. After step 10, B occurs 1749 times, C occurs 298 times, H occurs 161 times, and N occurs 865 times; taking the quantity of the most common element (B, 1749) and subtracting the quantity of the least common element (H, 161) produces 1749 - 161 = 1588.

Apply 10 steps of pair insertion to the polymer template and find the most and least common elements in the result. What do you get if you take the quantity of the most common element and subtract the quantity of the least common element?


In [ ]:
#load "../Shared/regexp.fsx"
open RegexpMatch

let testInput = """
NNCB

CH -> B
HH -> N
CB -> H
NH -> C
HB -> C
HC -> B
HN -> C
NN -> C
BH -> H
NC -> B
NB -> B
BN -> B
BB -> N
BC -> B
CC -> N
CN -> C
"""

let parse (input:string) =
    let parts = input.Replace("\r", "").Split("\n\n")
    let template = 
        parts[0].Split("\n")
        |> Seq.filter ( (<>) "" )
        |> Seq.head

    let rules =
        parts[1].Split("\n")
        |> Seq.filter ( (<>) "" )
        |> Seq.map (
            function
            | Regex @"(\w+)\s*->\s*(\w)" [pair; split] -> (pair, split)
            | token -> failwithf "Unknown input %s " token
            )
        |> Map.ofSeq

    (template, rules)

testInput |> parse

Item1,Item2
NNCB,"[ { [BB, N]: Key: BB, Value: N }, { [BC, B]: Key: BC, Value: B }, { [BH, H]: Key: BH, Value: H }, { [BN, B]: Key: BN, Value: B }, { [CB, H]: Key: CB, Value: H }, { [CC, N]: Key: CC, Value: N }, { [CH, B]: Key: CH, Value: B }, { [CN, C]: Key: CN, Value: C }, { [HB, C]: Key: HB, Value: C }, { [HC, B]: Key: HC, Value: B }, { [HH, N]: Key: HH, Value: N }, { [HN, C]: Key: HN, Value: C }, { [NB, B]: Key: NB, Value: B }, { [NC, B]: Key: NC, Value: B }, { [NH, C]: Key: NH, Value: C }, { [NN, C]: Key: NN, Value: C } ]"


In [ ]:
let splitTemplate (input:string) =
    (input + " ").ToCharArray()
    |> Seq.windowed 2
    |> Seq.map System.String

testInput |> parse |> fst |> splitTemplate

index,value
0,NN
1,NC
2,CB
3,B


In [ ]:
let runPolymerization (steps: int) (template:string) (rules:Map<string, string>) =
    let rec runStep (count: int) (state:seq<string * int64>) =
        if count=0 then 
            state
        else
            state
            |> Seq.map ( fun (pair, count) ->
                        if Map.containsKey pair rules then
                            let insert = rules[pair]
                            [
                                (pair.Substring(0,1) + insert, count)
                                (insert + pair.Substring(1,1), count)
                            ]
                        else
                            [pair, count]
                    )
            |> Seq.concat
            |> Seq.groupBy fst
            |> Seq.map (fun (key, pairs) -> 
                (key, pairs |> (Seq.sumBy snd) )
            )
            |> runStep (count-1)

    template
    |> splitTemplate
    |> Seq.map (fun key -> (key,1L))
    |> runStep steps

testInput |> parse ||> runPolymerization 3

index,Item1,Item2
0,NB,4
1,BB,4
2,BC,3
3,CN,2
4,NC,1
5,CC,1
6,BN,2
7,CH,2
8,HB,3
9,BH,1


In [ ]:
let evaluateQuantities (state: seq<string * int64>) =
    let ordered = 
        state
        |> Seq.map( fun (pair, count) ->
            [
                (pair[0], count)
            ]
        )
        |> Seq.concat
        |> Seq.groupBy fst
        |> Seq.map ( fun (key, items) -> 
            (key, items |> Seq.sumBy snd )
        )
        |> Array.ofSeq
        |> Array.sortBy snd

    (snd ordered[ordered.Length-1] ) - (snd ordered[0])

testInput |> parse ||> runPolymerization 10 |> evaluateQuantities

1588

In [ ]:
let task = System.IO.File.ReadAllText "task.txt"
task |> parse ||> runPolymerization 10 |> evaluateQuantities

4244

--- Part Two ---

The resulting polymer isn't nearly strong enough to reinforce the submarine. You'll need to run more steps of the pair insertion process; a total of 40 steps should do it.

In the above example, the most common element is B (occurring 2192039569602 times) and the least common element is H (occurring 3849876073 times); subtracting these produces 2188189693529.

Apply 40 steps of pair insertion to the polymer template and find the most and least common elements in the result. What do you get if you take the quantity of the most common element and subtract the quantity of the least common element?


In [ ]:
testInput |> parse ||> runPolymerization 40 |> evaluateQuantities

2188189693529

In [ ]:
task |> parse ||> runPolymerization 40 |> evaluateQuantities

4807056953866